# clean and validate data


In [2]:
%%sql
USE spark_catalog.dbo;

StatementMeta(, e162663a-ccea-4bca-ab84-114d5a01fe19, 4, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [3]:
%%sql
CREATE OR REPLACE TABLE silver_patients
USING DELTA
AS
SELECT
patient_id,
age,
CASE WHEN upper(trim(gender))='M' THEN 'male'
    WHEN upper(trim(gender))='F' THEN 'female'
        else 'N/A'
END AS gender,
comorbidity_index,
CASE WHEN comorbidity_index BETWEEN 0 AND 2 THEN 'low'
     WHEN comorbidity_index BETWEEN 3 AND 5 THEN 'average'
     WHEN comorbidity_index BETWEEN 6 AND 8 THEN 'high'
        ELSE 'N/A'
END AS level_of_risk,
admission_type,
baseline_risk_score,
los_hours AS length_of_stay,
deterioration_event,
deterioration_within_12h_from_admission,
CASE WHEN deterioration_hour=-1 THEN 0
 ELSE deterioration_hour
END AS deterioration_hour



  
FROM bronze_patients



StatementMeta(, e162663a-ccea-4bca-ab84-114d5a01fe19, 5, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [4]:
%%sql
UNCACHE TABLE silver_patients;
REFRESH TABLE silver_patients;

StatementMeta(, e162663a-ccea-4bca-ab84-114d5a01fe19, 7, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

In [5]:
%%sql
CREATE OR REPLACE TABLE silver_labs
USING DELTA
AS
SELECT patient_id,
hour_from_admission,
wbc_count,
lactate,
creatinine,
crp_level,
hemoglobin,
sepsis_risk_score,
CASE WHEN max_wbc_count > 12 THEN 1
     WHEN max_wbc_count < 7 THEN 1
     ELSE 0
END AS check_wbc_count,
CASE WHEN max_lactate > 3 THEN 1
     WHEN max_lactate < 1.5 THEN 1
     ELSE 0
END AS check_lactate,
CASE WHEN max_creatinine > 1.5 THEN 1
     ELSE 0
END AS check_creatine,
CASE WHEN max_crp_level > 55 THEN 1
     WHEN max_crp_level < 20 THEN 1
     ELSE 0
END AS check_crp_level,
CASE WHEN max_hemoglobin > 12.5 THEN 1
     WHEN max_hemoglobin < 10 THEN 1
     ELSE 0
END AS check_hemoglobin 

FROM
(SELECT
    *,
    MAX(wbc_count) OVER (PARTITION BY patient_id ) AS max_wbc_count,
    MAX(lactate) OVER (PARTITION BY patient_id ) AS max_lactate,
    MAX(creatinine) OVER (PARTITION BY patient_id ) AS max_creatinine,
    MAX(crp_level) OVER (PARTITION BY patient_id ) AS max_crp_level,
    MAX(hemoglobin) OVER (PARTITION BY patient_id ) AS max_hemoglobin
    

   
FROM spark_catalog.dbo.bronze_labs) ;


StatementMeta(, e162663a-ccea-4bca-ab84-114d5a01fe19, 8, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [6]:
%%sql
UNCACHE TABLE silver_labs;
REFRESH TABLE silver_labs;


StatementMeta(, e162663a-ccea-4bca-ab84-114d5a01fe19, 10, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

In [7]:
%%sql
CREATE OR REPLACE TABLE silver_vitals
USING DELTA 
AS
SELECT patient_id,
hour_from_admission,
heart_rate,
respiratory_rate,
spo2_pct,
temperature_c,
systolic_bp,
diastolic_bp,
oxygen_device,
oxygen_flow,
mobility_score,
nurse_alert,
CASE WHEN heart_rate > 150  THEN 'high'
     WHEN heart_rate < 60 THEN 'low'
     ELSE 'normal'
END AS status_of_heart_rate,
CASE WHEN respiratory_rate > 20 THEN 'high'
     WHEN respiratory_rate < 12 THEN 'low'
     ELSE 'normal'
     
END AS status_of_respiratory_rate,
CASE WHEN spo2_pct > 94 THEN 'normal'
     WHEN spo2_pct < 94 THEN 'low'
     
END AS check_spo2_pct,
CASE WHEN temperature_c > 37.5 THEN 'high'
     WHEN temperature_c < 36 THEN 'low'
    ELSE 'normal'
     
END AS check_temperature_c,
CASE WHEN systolic_bp > 115 THEN 'high'
     WHEN systolic_bp < 85 THEN 'low'
     ELSE 'normal'
END AS check_systolic_bp,
CASE WHEN diastolic_bp > 80 THEN 'high'
     WHEN diastolic_bp < 55 THEN 'low'
     ELSE 'normal'
END AS check_diastolic_bp,
MAX(heart_rate) OVER (PARTITION BY patient_id ORDER BY hour_from_admission ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS max_heart_rate,
MAX(respiratory_rate) OVER (PARTITION BY patient_id ORDER BY hour_from_admission ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW ) AS max_respiratory_rate,
MAX(spo2_pct) OVER (PARTITION BY patient_id ORDER BY hour_from_admission ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW ) AS max_spo2_pct,
MAX(temperature_c) OVER (PARTITION BY patient_id ORDER BY hour_from_admission ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW ) AS max_temperature_c,
MAX(systolic_bp) OVER (PARTITION BY patient_id ORDER BY hour_from_admission ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW ) AS max_systolic_bp,
MAX(diastolic_bp) OVER (PARTITION BY patient_id ORDER BY hour_from_admission ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW ) AS max_diastolic_bp


FROM bronze_vitals

StatementMeta(, e162663a-ccea-4bca-ab84-114d5a01fe19, 11, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [8]:
%%sql
UNCACHE TABLE silver_vitals;
REFRESH TABLE silver_vitals;


StatementMeta(, e162663a-ccea-4bca-ab84-114d5a01fe19, 13, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>